In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movierecommenderdataset/movies.csv
/kaggle/input/movierecommenderdataset/ratings.csv


In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv('/kaggle/input/movierecommenderdataset/movies.csv')
ratings = pd.read_csv('/kaggle/input/movierecommenderdataset/ratings.csv')

In [4]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [7]:
df = movies.merge(ratings, how='left', on='movieId')

In [8]:
df.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [9]:
df.shape

(100854, 6)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100854 entries, 0 to 100853
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100854 non-null  int64  
 1   title      100854 non-null  object 
 2   genres     100854 non-null  object 
 3   userId     100836 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  100836 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 5.4+ MB


In [13]:
df.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [14]:
# replace the missing value from the dataset
df.dropna(inplace=True)

In [16]:
rating_counts = pd.DataFrame(df['title'].value_counts())
rating_counts

,title
Forrest Gump (1994),329
"Shawshank Redemption, The (1994)",317
Pulp Fiction (1994),307
"Silence of the Lambs, The (1991)",279
"Matrix, The (1999)",278
...,...
We're Back! A Dinosaur's Story (1993),1
American Hardcore (2006),1
Shanghai Surprise (1986),1
Let's Get Harry (1986),1


In [17]:


# select movies which have less then 50 ratings
rare_movies = rating_counts[rating_counts['title'] <= 50].index

# find the common movies
common_movies = df[~df['title'].isin(rare_movies)]

# pivot to form the matrix for similarity calculation
user_movie_df = common_movies.pivot_table('rating', 'userId', 'title')

user_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Wild Wild West (1999),Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,5.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=20).values)

random_user_df = user_movie_df[user_movie_df.index == random_user]

print(random_user_df)

movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()


len(movies_watched)

title   10 Things I Hate About You (1999)  12 Angry Men (1957)  \
userId                                                           
121.0                                 NaN                  NaN   

title   2001: A Space Odyssey (1968)  28 Days Later (2002)  300 (2007)  \
userId                                                                   
121.0                            NaN                   NaN         NaN   

title   40-Year-Old Virgin, The (2005)  A.I. Artificial Intelligence (2001)  \
userId                                                                        
121.0                              NaN                                  NaN   

title   Abyss, The (1989)  Ace Ventura: Pet Detective (1994)  \
userId                                                         
121.0                 NaN                                3.0   

title   Ace Ventura: When Nature Calls (1995)  ...  Wild Wild West (1999)  \
userId                                         ...                    

47

In [19]:
movies_watched_df = user_movie_df[movies_watched]

user_movie_count = movies_watched_df.T.notnull().sum()

user_movie_count = user_movie_count.reset_index()

In [20]:
user_movie_count.columns = ['userId', 'movie_count']

user_movie_count.head()

,userId,movie_count
0,1.0,14
1,2.0,1
2,3.0,1
3,4.0,5
4,5.0,20


In [21]:
users_same_movies = user_movie_count[user_movie_count['movie_count'] > 20]['userId']

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)], 
                     random_user_df[movies_watched]])

final_df.head()

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),"American President, The (1995)",Apollo 13 (1995),Batman (1989),Beauty and the Beast (1991),"Birdcage, The (1996)",Braveheart (1995),Broken Arrow (1996),Casino (1995),...,Schindler's List (1993),Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)",Sleepless in Seattle (1993),Speed (1994),Stargate (1994),Toy Story (1995),True Lies (1994),True Romance (1993),What's Eating Gilbert Grape (1993)
userId,,,,,,,,,,,,,,,,,,,,,
6.0,3.0,2.0,4.0,4.0,3.0,5.0,4.0,5.0,4.0,4.0,...,3.0,4.0,5.0,3.0,5.0,5.0,NaN,4.0,NaN,5.0
8.0,NaN,NaN,4.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,...,5.0,4.0,5.0,3.0,4.0,NaN,NaN,5.0,NaN,NaN
14.0,2.0,1.0,NaN,4.0,2.0,NaN,NaN,5.0,5.0,NaN,...,3.0,5.0,3.0,4.0,3.0,NaN,NaN,NaN,NaN,5.0
18.0,2.5,NaN,NaN,NaN,NaN,3.0,NaN,4.5,NaN,4.5,...,4.5,4.5,5.0,NaN,2.5,NaN,3.5,2.5,4.0,4.0
19.0,2.0,2.0,NaN,NaN,5.0,NaN,1.0,NaN,NaN,NaN,...,NaN,3.0,NaN,3.0,4.0,2.0,4.0,3.0,NaN,NaN


In [22]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

corr_df.head()

userId  userId
56.0    604.0    -0.819092
580.0   43.0     -0.810163
524.0   483.0    -0.749797
483.0   468.0    -0.670774
133.0   604.0    -0.666667
dtype: float64

In [ ]:
# Next steps, filter the rows where userId1 = random_user_id
# Sort based on the column
# Find top 50 similar users
# Take average rating of every movie by all the users.
# Recommend the movies.